### verificar preprocessador

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

#xgboost
from xgboost import XGBClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_tic import preprocessador_titanic
from src.plot_metrica_class import *


BASE = Path.cwd().parent
DATA_DIR = BASE/"data"/"raw"
X       = pd.read_csv(DATA_DIR/"train.csv")
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo finalizado em: 14:35:19


In [3]:
#função basica
def vizualizar_transformacao(PP,X):
    dfX = PP.transform(X)

    
    print("═" * 70)
    print("DETALHAMENTO DAS NOVAS COLUNAS CRIADAS")
    print("═" * 70)

# Identificar novas colunas
    original_cols = X.columns.tolist() if hasattr(X, 'columns') else []
    transformed_cols = dfX.columns.tolist() if hasattr(dfX, 'columns') else []
    new_cols = [col for col in transformed_cols if col not in original_cols]

    print(f"\n📈 RESUMO ESTATÍSTICO:")
    print(f"   • Dimensão originais: {X.shape}")
    print(f"   • Colunas originais: {len(original_cols)}")
    print(f"   • Colunas após transformação: {len(transformed_cols)}")
    print(f"   • Novas colunas criadas: {len(new_cols)}")
    print(f"   • Aumento: {(len(new_cols)/len(original_cols)*100):.1f}%")

    if new_cols:
        print(f"\n📋 LISTA DAS NOVAS COLUNAS ({len(new_cols)} no total):")
        print("-" * 50)
        
        # Exibir em colunas para melhor visualização
        for i in range(0, len(new_cols), 3):
            line_cols = new_cols[i:i+3]
            col_str = "   ".join([f"{col:<25}" for col in line_cols])
            print(f"   {col_str}")
        
        print(f"\n🔍 DETALHES DAS NOVAS COLUNAS:")
        print("-" * 50)
        
        for col in new_cols[:len(new_cols)]:
            unique_vals = dfX[col].nunique() if hasattr(dfX[col], 'nunique') else 'N/A'
            dtype = dfX[col].dtype
            non_null = dfX[col].count() if hasattr(dfX[col], 'count') else 'N/A'
            total = len(dfX)
            
            print(f"   • {col} {dtype}")
            print(f"      Valores únicos: {unique_vals}")
            print(f"      Não nulos: {non_null}/{total} ({(non_null/total*100):.1f}%)")
            
            # Mostrar valores únicos se forem poucos
            if hasattr(dfX[col], 'unique') and unique_vals <= 5:
                vals = dfX[col].unique()[:5]
                print(f"     Valores: {vals}")
            print()
        
    else:
        print("\nℹ️  Nenhuma nova coluna criada - transformações aplicadas in-place.")
    
    print(f"\n👁️  VISUALIZAÇÃO (primeiras 5 linhas):")
    display(dfX.head(5))
    return dfX

In [8]:
X.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
BASE = Path.cwd().parent

PP1 = joblib.load(BASE/'src'/'preprocess_Titanic_v1.2.joblib')['preprocessador']


out1=vizualizar_transformacao(PP1,X)

══════════════════════════════════════════════════════════════════════
DETALHAMENTO DAS NOVAS COLUNAS CRIADAS
══════════════════════════════════════════════════════════════════════

📈 RESUMO ESTATÍSTICO:
   • Dimensão originais: (891, 12)
   • Colunas originais: 12
   • Colunas após transformação: 25
   • Novas colunas criadas: 21
   • Aumento: 175.0%

📋 LISTA DAS NOVAS COLUNAS (21 no total):
--------------------------------------------------
   HasCabin                    FamilySize                  Age2                     
   Sex_female                  Sex_male                    Embarked_C               
   Embarked_Q                  Embarked_S                  Deck_A                   
   Deck_B                      Deck_C                      Deck_D                   
   Deck_E                      Deck_F                      Deck_G                   
   Deck_U                      Title_Master                Title_Miss               
   Title_Mr                    Title_Mrs   

,Pclass,SibSp,Parch,Fare,HasCabin,FamilySize,Age2,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_U,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,3,1,0,7.2500,0,2,22.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False
1,1,1,0,71.2833,1,2,38.0,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
2,3,0,0,7.9250,0,1,26.0,True,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False
3,1,1,0,53.1000,1,2,35.0,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False
4,3,0,0,8.0500,0,1,35.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False
